<a href="https://colab.research.google.com/github/susanchen475/mgmt467-analytics-portfolio/blob/main/Lab2_Advanced_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt467-lab' # This is derived from your provided table name
dataset_id = 'superstore_data'
table_id = 'sales'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt467-lab.superstore_data.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `mgmt467-lab.superstore_data.sales`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's query the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a query to select from the new view
    query_string = f"""
    SELECT
      *
    FROM
      `{project_id}.{dataset_id}.{new_view_id}`
    LIMIT 10;
    """

    # Execute the query
    query_job = client.query(query_string)

    # Fetch the results into a DataFrame
    results_df = query_job.to_dataframe()

    # Display the DataFrame
    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        display(results_df)

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")

View 'superstore_clean' created/replaced successfully in dataset 'superstore_data'.

--- First 10 rows from the new view 'superstore_clean' ---


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,5769,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,3.15,1,0.0,1.5120
1,5770,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-PA-10002377,Office Supplies,Paper,Adams Telephone Message Book W/Dividers/Space ...,22.72,4,0.0,10.2240
2,9028,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10002597,Furniture,Furnishings,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,3,0.0,6.2244
3,9029,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,3,0.0,61.3824
4,8332,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-ST-10004634,Office Supplies,Storage,"Personal Folder Holder, Ebony",11.21,1,0.0,3.3630
5,8333,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,FUR-CH-10002647,Furniture,Chairs,"Situations Contoured Folding Chairs, 4/Set",354.90,5,0.0,88.7250
6,8334,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-PA-10001801,Office Supplies,Paper,Xerox 193,17.94,3,0.0,8.7906
7,8335,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-BI-10004632,Office Supplies,Binders,GBC Binding covers,51.80,4,0.0,23.3100
8,526,CA-2015-158792,2015-12-26,2016-01-02,Standard Class,BD-11605,Brian Dahlen,Consumer,United States,Lawrence,...,1841,East,OFF-FA-10002815,Office Supplies,Fasteners,Staples,22.20,5,0.0,10.4340
9,1312,CA-2016-141082,2016-12-09,2016-12-13,Standard Class,FM-14380,Fred McMath,Consumer,United States,Lawrence,...,1841,East,OFF-LA-10001404,Office Supplies,Labels,Avery 517,3.69,1,0.0,1.7343


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt467-lab.superstore_data.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** The result matches with the expectation, and the only difference was selecting sub_category. Since it is using the cleaned data, it has being transferred from Sub-Category to the correct format of Sub_Category.

In [ ]:
query_string = """
SELECT
  DISTINCT sub_category
FROM
  `mgmt467-lab.superstore_data.superstore_clean`
WHERE
  Region = 'West'
ORDER BY
  sub_category ASC
LIMIT 100;
"""

# Execute the query
query_job = client.query(query_string)

# Fetch the results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

,sub_category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = """
SELECT
    customer_id,
    SUM(profit) AS total_profit
FROM
    `mgmt467-lab.superstore_data.superstore_clean`
GROUP BY
    customer_id
ORDER BY
    total_profit DESC
LIMIT 10;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,customer_id,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
query_string = """
SELECT
    ship_mode,
    COUNT(*) AS order_count
FROM
    `mgmt467-lab.superstore_data.superstore_clean`
WHERE
    Category = 'Technology'
GROUP BY
    ship_mode
ORDER BY
    order_count DESC;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,ship_mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `mgmt467-lab.superstore_data.superstore_clean`
Assume: `order_date` is a DATE or TIMESTAMP column named exactly `order_date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string = """
SELECT
    FORMAT_DATE('%Y-%m', order_date) AS year_month,
    SUM(Sales) AS monthly_revenue
FROM
    `mgmt467-lab.superstore_data.superstore_clean`
WHERE
    order_date >= '2016-09-05'
    AND order_date <= '2017-12-09'
    AND order_date >= DATE_TRUNC(DATE_SUB(PARSE_DATE('%Y-%m-%d', '2017-12-09'), INTERVAL 13 MONTH), MONTH)
    AND order_date < DATE_TRUNC(PARSE_DATE('%Y-%m-%d', '2017-12-09'), MONTH)
GROUP BY
    year_month
ORDER BY
    year_month ASC
LIMIT 1000;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,year_month,monthly_revenue
0,2016-11,79411.9658
1,2016-12,96999.0430
2,2017-01,43971.3740
3,2017-02,20301.1334
4,2017-03,58872.3528
5,2017-04,36521.5361
6,2017-05,44261.1102
7,2017-06,52981.7257
8,2017-07,45264.4160
9,2017-08,63120.8880


The query is unable to generate data because the

### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [ ]:
query_string = """
SELECT
    Sub_Category,
    SUM(Profit) AS total_profit
FROM
    `mgmt467-lab.superstore_data.superstore_clean`
GROUP BY
    Sub_Category
HAVING
    SUM(Profit) < 0
ORDER BY
    total_profit ASC;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

#The HAVING clause is used instead of WHERE because HAVING filters results after aggregation (in this case, SUM(Profit)), while WHERE filters rows before aggregation.

,Sub_Category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
query_string = """
WITH ProductCTE AS (
    SELECT DISTINCT
        Product_ID,
        CONCAT('Product_Name_', Product_ID) AS Product_Name
    FROM
        `mgmt467-lab.superstore_data.superstore_clean`
)
SELECT
    s.Product_ID,
    p.Product_Name,
    SUM(s.Sales) AS total_sales
FROM
    `mgmt467-lab.superstore_data.superstore_clean` AS s
JOIN
    ProductCTE AS p
ON
    s.Product_ID = p.Product_ID
GROUP BY
    s.Product_ID,
    p.Product_Name
ORDER BY
    total_sales DESC;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Product_ID,Product_Name,total_sales
0,TEC-CO-10004722,Product_Name_TEC-CO-10004722,61599.824
1,OFF-BI-10003527,Product_Name_OFF-BI-10003527,27453.384
2,TEC-MA-10002412,Product_Name_TEC-MA-10002412,22638.480
3,FUR-CH-10002024,Product_Name_FUR-CH-10002024,21870.576
4,OFF-BI-10001359,Product_Name_OFF-BI-10001359,19823.479
...,...,...,...
1857,OFF-AR-10003986,Product_Name_OFF-AR-10003986,7.700
1858,OFF-EN-10001535,Product_Name_OFF-EN-10001535,7.072
1859,OFF-PA-10000048,Product_Name_OFF-PA-10000048,6.480
1860,OFF-LA-10003388,Product_Name_OFF-LA-10003388,5.760


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

This query aims to identify the top-performing states within each sales region based on total sales. First, we calculate the total sales for each state within each region using a Common Table Expression (CTE) called state_sales. This CTE groups the raw sales data by Region and State, summing the Sales for each combination. Next, we create a second CTE, ranked_state_sales, to assign a rank to each state within its respective region. This is achieved using the RANK() window function, which partitions the data by Region and orders it by total_sales in descending order, assigning a rank (1 being the highest sales). Finally, the main SELECT statement filters these ranked results to include only those states where the sales_rank is 3 or less, effectively giving us the top 3 states by sales for each region, and then sorts the final output by Region and sales_rank.

In [ ]:
query_string = """
WITH state_sales AS (
    SELECT
        Region,
        State,
        SUM(Sales) AS total_sales
    FROM
        `mgmt467-lab.superstore_data.superstore_clean`
    GROUP BY
        Region,
        State
), ranked_state_sales AS (
    SELECT
        Region,
        State,
        total_sales,
        RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
    FROM
        state_sales
)
SELECT
    Region,
    State,
    total_sales,
    sales_rank
FROM
    ranked_state_sales
WHERE
    sales_rank <= 3
ORDER BY
    Region,
    sales_rank;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Region,State,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

Missing Year Data: A sub-category might have sales in one year but not the other (e.g., sales in 2024 but not in 2023, or vice-versa). If we only use an INNER JOIN between the 2023 and 2024 sales data, we would exclude any sub-category that didn't have sales in both years. This would incorrectly remove sub-categories that actually had a significant increase (or decrease) from zero.

* Handling: Use a FULL OUTER JOIN and the COALESCE() function. A FULL OUTER JOIN ensures that all sub-categories present in either the 2023 or 2024 sales data are included in the result. COALESCE() is then used to replace NULL values (which occur when a sub-category is missing in one of the years) with 0. This allows us to correctly calculate the delta, even if one of the years had no sales for a particular sub-category. For example, if a sub-category had $0 sales in 2023 and $500 in 2024, COALESCE(s24.sales_2024, 0) - COALESCE(s23.sales_2023, 0) would correctly result in 500 - 0 = 500.

Sub-categories with Zero or Negative YoY Delta: The original prompt asked for the largest YoY increase, implying positive deltas. However, the FULL OUTER JOIN and COALESCE logic will also produce rows for sub-categories where sales decreased or stayed the same (resulting in a negative or zero yoy_delta). If the explicit goal is only to see increases, we need to filter these out.

* Handling: Add a WHERE clause after the join to explicitly filter for positive yoy_delta. This ensures that only sub-categories that actually grew in revenue from 2023 to 2024 are included in the final output, fulfilling the "largest YoY revenue increase" requirement. The condition WHERE (COALESCE(s24.sales_2024, 0) - COALESCE(s23.sales_2023, 0)) > 0 achieves this.

In [ ]:
query_string = """
WITH yr_sales AS (
    SELECT
        sub_category,
        EXTRACT(YEAR FROM order_date) AS year,
        SUM(sales) AS total_sales
    FROM
        `mgmt467-lab.superstore_data.superstore_clean`
    WHERE
        EXTRACT(YEAR FROM order_date) IN (2023, 2024)
    GROUP BY
        sub_category,
        year
),
sales_2023 AS (
    SELECT
        sub_category,
        total_sales AS sales_2023
    FROM
        yr_sales
    WHERE
        year = 2023
),
sales_2024 AS (
    SELECT
        sub_category,
        total_sales AS sales_2024
    FROM
        yr_sales
    WHERE
        year = 2024
)
SELECT
    COALESCE(s23.sub_category, s24.sub_category) AS sub_category,
    s23.sales_2023,
    s24.sales_2024,
    (COALESCE(s24.sales_2024, 0) - COALESCE(s23.sales_2023, 0)) AS yoy_delta
FROM
    sales_2023 AS s23
FULL OUTER JOIN
    sales_2024 AS s24
ON
    s23.sub_category = s24.sub_category
WHERE
    (COALESCE(s24.sales_2024, 0) - COALESCE(s23.sales_2023, 0)) > 0
ORDER BY
    yoy_delta DESC
LIMIT 5;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,sub_category,sales_2023,sales_2024,yoy_delta


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.
* ROW_NUMBER() assigns a unique, sequential integer to each row within a partition, ensuring that even if there are ties in total_sales, each row gets a distinct rank. RANK() (or DENSE_RANK()) would assign the same rank to rows with identical total_sales values, which might result in more than one sub-category being returned per region if there are ties for the highest revenue.

In [ ]:
query_string = """
WITH subcat_sales AS (
    SELECT
        Region,
        Sub_Category,
        SUM(Sales) AS total_sales
    FROM
        `mgmt467-lab.superstore_data.superstore_clean`
    GROUP BY
        Region,
        Sub_Category
),
ranked_subcat_sales AS (
    SELECT
        Region,
        Sub_Category,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS rn
    FROM
        subcat_sales
)
SELECT
    Region,
    Sub_Category,
    total_sales
FROM
    ranked_subcat_sales
WHERE
    rn = 1
ORDER BY
    Region;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Region,Sub_Category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
query_string = """
WITH YearlyRevenue AS (
    SELECT
        EXTRACT(YEAR FROM Order_Date) AS year,
        SUM(sales) AS yearly_revenue
    FROM
        `mgmt467-lab.superstore_data.superstore_clean`
    WHERE
        sub_category = 'Phones'
    GROUP BY
        year
),
LaggedRevenue AS (
    SELECT
        year,
        yearly_revenue,
        LAG(yearly_revenue, 1, 0) OVER (ORDER BY year) AS prev_revenue
    FROM
        YearlyRevenue
)
SELECT
    year,
    yearly_revenue,
    prev_revenue,
    CASE
        WHEN prev_revenue = 0 THEN NULL -- Handle cases where previous year had zero revenue
        ELSE 100.0 * (yearly_revenue - prev_revenue) / prev_revenue
    END AS yoy_pct
FROM
    LaggedRevenue
ORDER BY
    year ASC;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,0.000,NaN
1,2015,68313.702,77390.806,-11.728918
2,2016,78962.030,68313.702,15.587397
3,2017,105340.516,78962.030,33.406545


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).
* While developing, add a WHERE order_date >= '2024-01-01' to limit scanned data.

In [ ]:
#For exploration, consider limiting the date range to reduce costs, e.g., WHERE Order_Date BETWEEN '2023-01-01' AND '2023-12-31'
query_string = """
WITH monthly_sales AS (
    SELECT
        DATE_TRUNC(order_date, MONTH) AS month,
        SUM(sales) AS monthly_revenue
    FROM
        `mgmt467-lab.superstore_data.superstore_clean`
    WHERE
        Segment = 'Corporate'
    GROUP BY
        month
)
SELECT
    month,
    monthly_revenue,
    AVG(monthly_revenue) OVER (
        ORDER BY month
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS ma_3
FROM monthly_sales
ORDER BY month ASC;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,month,monthly_revenue,ma_3
0,2014-01-01,1701.5280,1701.528000
1,2014-02-01,1183.6680,1442.598000
2,2014-03-01,11106.7990,4663.998333
3,2014-04-01,14131.7290,8807.398667
4,2014-05-01,9142.0000,11460.176000
5,2014-06-01,3970.9140,9081.547667
6,2014-07-01,10032.9880,7715.300667
7,2014-08-01,7451.7740,7151.892000
8,2014-09-01,15507.7450,10997.502333
9,2014-10-01,12637.6780,11865.732333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [ ]:
query_string = """
-- Query 1: Slice by Region and Time
SELECT
    Region,
    EXTRACT(YEAR FROM Order_Date) AS sales_year,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    SUM(Discount) AS total_discount
FROM
    `mgmt467-lab.superstore_data.superstore_clean`
WHERE
    Sub_Category = 'Tables'
GROUP BY
    Region,
    sales_year
ORDER BY
    Region,
    sales_year;

-- Query 2: Control for Ship Mode
SELECT
    Ship_Mode,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    AVG(Discount) AS average_discount
FROM
    `mgmt467-lab.superstore_data.superstore_clean`
WHERE
    Sub_Category = 'Tables'
GROUP BY
    Ship_Mode
ORDER BY
    Ship_Mode;
    """

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Ship_Mode,total_sales,total_profit,average_discount
0,First Class,28800.7760,-1365.3665,0.240426
1,Same Day,9644.3470,-1129.4225,0.261905
2,Second Class,43693.7475,-3320.6799,0.248361
3,Standard Class,124826.6615,-11910.0122,0.270526


The primary comparison strategy involves seeking exceptions to the "negative-profit" rule within the derived segments. From Query 1, the finding is nuanced if any specific combination of Region and sales_year shows a positive SUM(Profit). If profitability is positive in certain localized periods, the initial conclusion that the entire sub-category is inherently unprofitable due to discounting is false, suggesting the issue is temporal or regional cost variation, not universal discounting policy. Simultaneously, Query 2 allows for the assessment of operational overhead: if the majority of the negative profit is concentrated in high-urgency, high-cost segments like 'Same Day' Ship Mode, and these modes do not necessarily correlate with the highest average_discount, the root cause shifts from simple discounting to complex logistics and fulfillment costs, thereby fundamentally nuancing the initial finding.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---